In [1]:
# -*- coding: utf-8 -*-
import os
import pickle
from typing import List

import pandas
import numpy as np
from tqdm import tqdm

In [2]:
orig_name = 'Dataset II-1.csv'
csv_name = 'II-1'

In [3]:
'''
读 csv
'''
assert os.path.exists('../../data')
dataset = pandas.read_csv('../../data/Dataset II (mismatch-only)/%s' % orig_name, delimiter=',')
dataset

,MutatedSequence,Annotation,Category,Day21-ETP,Position,Protein Annotation,TranscriptID,WTSequence,MutatedSequence.1,30mer,30mer_mut,Day21-ETP-binarized
0,ACCGAGGATTGAGTAGTCCT,"G:T,9",Mismatch,0.999571,9.0,52.47,ENST00000262262,ACCGAGGAGTGAGTAGTCCT,ACCGAGGATTGAGTAGTCCT,ACCGAGGAGTGAGTAGTCCTGGG,ACCGAGGATTGAGTAGTCCTGGG,1.0
1,ACTGAGGAGTGAGTAGTCCT,"C:T,3",Mismatch,0.989121,3.0,52.47,ENST00000262262,ACCGAGGAGTGAGTAGTCCT,ACTGAGGAGTGAGTAGTCCT,ACCGAGGAGTGAGTAGTCCTGGG,ACTGAGGAGTGAGTAGTCCTGGG,1.0
2,AGCGAGGAGTGAGTAGTCCT,"C:G,2",Mismatch,0.914495,2.0,52.47,ENST00000262262,ACCGAGGAGTGAGTAGTCCT,AGCGAGGAGTGAGTAGTCCT,ACCGAGGAGTGAGTAGTCCTGGG,AGCGAGGAGTGAGTAGTCCTGGG,1.0
3,ACCGAGGAGTGAGTAGTCCA,"T:A,20",Mismatch,0.908142,20.0,52.47,ENST00000262262,ACCGAGGAGTGAGTAGTCCT,ACCGAGGAGTGAGTAGTCCA,ACCGAGGAGTGAGTAGTCCTGGG,ACCGAGGAGTGAGTAGTCCAGGG,1.0
4,ACAGAGGAGTGAGTAGTCCT,"C:A,3",Mismatch,0.903949,3.0,52.47,ENST00000262262,ACCGAGGAGTGAGTAGTCCT,ACAGAGGAGTGAGTAGTCCT,ACCGAGGAGTGAGTAGTCCTGGG,ACAGAGGAGTGAGTAGTCCTGGG,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4848,GTCCCTGGATATAATGGCTC,TT,PAM,0.000022,-1.0,17.86,ENST00000262262,GTCCCTGGATATAATGGCTC,GTCCCTGGATATAATGGCTC,GTCCCTGGATATAATGGCTCCGG,GTCCCTGGATATAATGGCTCCTT,0.0
4849,CAAAAACCTGACCTGCTCTG,GT,PAM,0.000016,-1.0,45.05,ENST00000262262,CAAAAACCTGACCTGCTCTG,CAAAAACCTGACCTGCTCTG,CAAAAACCTGACCTGCTCTGTGG,CAAAAACCTGACCTGCTCTGTGT,0.0
4850,CCGCCTCCTTGGGGATCCCA,TA,PAM,0.000012,-1.0,26.37,ENST00000262262,CCGCCTCCTTGGGGATCCCA,CCGCCTCCTTGGGGATCCCA,CCGCCTCCTTGGGGATCCCAGGG,CCGCCTCCTTGGGGATCCCAGTA,0.0
4851,ATTATCCCTCCTCCTGGCGT,TA,PAM,0.000007,-1.0,29.67,ENST00000262262,ATTATCCCTCCTCCTGGCGT,ATTATCCCTCCTCCTGGCGT,ATTATCCCTCCTCCTGGCGTCGG,ATTATCCCTCCTCCTGGCGTCTA,0.0


In [4]:
on = dataset['30mer']
off = dataset['30mer_mut']
val = dataset['Day21-ETP-binarized']
on,off,val

(0       ACCGAGGAGTGAGTAGTCCTGGG
 1       ACCGAGGAGTGAGTAGTCCTGGG
 2       ACCGAGGAGTGAGTAGTCCTGGG
 3       ACCGAGGAGTGAGTAGTCCTGGG
 4       ACCGAGGAGTGAGTAGTCCTGGG
                  ...           
 4848    GTCCCTGGATATAATGGCTCCGG
 4849    CAAAAACCTGACCTGCTCTGTGG
 4850    CCGCCTCCTTGGGGATCCCAGGG
 4851    ATTATCCCTCCTCCTGGCGTCGG
 4852    GCAGTTGTTCCTACTGGGATCGG
 Name: 30mer, Length: 4853, dtype: object,
 0       ACCGAGGATTGAGTAGTCCTGGG
 1       ACTGAGGAGTGAGTAGTCCTGGG
 2       AGCGAGGAGTGAGTAGTCCTGGG
 3       ACCGAGGAGTGAGTAGTCCAGGG
 4       ACAGAGGAGTGAGTAGTCCTGGG
                  ...           
 4848    GTCCCTGGATATAATGGCTCCTT
 4849    CAAAAACCTGACCTGCTCTGTGT
 4850    CCGCCTCCTTGGGGATCCCAGTA
 4851    ATTATCCCTCCTCCTGGCGTCTA
 4852    GCAGTTGTTCCTACTGGGATCCC
 Name: 30mer_mut, Length: 4853, dtype: object,
 0       1.0
 1       1.0
 2       1.0
 3       1.0
 4       1.0
        ... 
 4848    0.0
 4849    0.0
 4850    0.0
 4851    0.0
 4852    0.0
 Name: Day21-ETP-binarized, Length: 4853,

In [5]:
tlen =24
ons = []
offs = []
for i in range(len(on)):
    ons.append('-'*(tlen-len(on[i]))+on[i])
for i in range(len(off)):
    offs.append('-'*(tlen-len(off[i]))+off[i])

In [6]:
min([len(s) for s in ons])

24

In [7]:
for i in tqdm(range(len(ons))):
    for j in range(tlen):
        if ons[i][j]=='N':
            ons[i][j]=offs[i][j]

df = pandas.DataFrame(data={
    'on':ons,
    'off':offs,
    'val':val
})
df.to_csv('../../data/%s.csv' % csv_name, index=False)

100%|██████████| 4853/4853 [00:00<00:00, 285759.81it/s]